## Setup

In [ ]:
!pip install scikit-fuzzy
!pip install EasyGA --no-deps

import numpy as np
import pandas as pd
import random

import skfuzzy as fuzz
from skfuzzy import control as ctrl

import EasyGA

## Load and Clip Data

In [ ]:
train_df = pd.read_csv("tipper_train.csv")
test_df  = pd.read_csv("tipper_test.csv")

input_cols = ["food temperature", "food flavor", "portion size", "attentiveness", "friendliness", "speed of service"]

# clip inputs to [0,1]
train_df[input_cols] = train_df[input_cols].clip(0, 1)
test_df[input_cols] = test_df[input_cols].clip(0, 1)

# scale tip to [0,30] if it was in [0,1]
if train_df["tip"].max() <= 1.5:
    train_df["tip"] *= 30.0
    test_df["tip"] *= 30.0

# convert one df row into fuzzy inputs on [0,10]
def row_to_inputs(row):
    return {
        "temperature":float(row["food temperature"] *10.0),
        "flavor": float(row["food flavor"] * 10.0),
        "portion_size": float(row["portion size"] * 10.0),
        "attentiveness": float(row["attentiveness"] * 10.0),
        "friendliness": float(row["friendliness"] * 10.0),
        "speed": float(row["speed of service"] * 10.0)
    }

## Membership functions, setup_fuzzy_system, evaluation helpers

In [ ]:
VAR_NAMES = ["temperature","flavor","portion_size","attentiveness","friendliness","speed","food_quality","service_quality","food_quality_for_tip","service_quality_for_tip","tip"]

CHROMOSOME_LENGTH = len(VAR_NAMES) * 3  # 3 genes per variable

def extract_gene_values(chromosome):
    if hasattr(chromosome, "genes"):
        return [g.value for g in chromosome.genes]
    else:
        return list(chromosome)

# build three triangular memberships functions from 3 gene values
def three_tri_from_genes(var, g1, g2, g3):
    def get_value(g):
        if hasattr(g, 'value'):
            return float(g.value)
        return float(g)

    g1, g2, g3 = get_value(g1), get_value(g2), get_value(g3)

    umin, umax = var.universe.min(), var.universe.max()

    span = umax - umin

    genes = sorted([g1, g2, g3])

    c1 = umin + genes[0] * span
    c2 = umin + genes[1] * span
    c3 = umin + genes[2] * span

    var["poor"] = fuzz.trimf(var.universe, [umin, umin, c2])
    var["average"] = fuzz.trimf(var.universe, [c1, c2, c3])
    var["good"] = fuzz.trimf(var.universe, [c2, umax, umax])

# build full fuzzy tree
def setup_fuzzy_system(chromosome):
    chromosome = extract_gene_values(chromosome)

    x = np.linspace(0, 10, 11)
    x_tip = np.linspace(0, 30, 31)

    temperature = ctrl.Antecedent(x, "temperature")
    flavor = ctrl.Antecedent(x, "flavor")
    portion_size = ctrl.Antecedent(x, "portion_size")
    attentiveness = ctrl.Antecedent(x, "attentiveness")
    friendliness = ctrl.Antecedent(x, "friendliness")
    speed = ctrl.Antecedent(x, "speed")

    food_quality = ctrl.Consequent(x, "food_quality")
    service_quality = ctrl.Consequent(x, "service_quality")

    food_quality_for_tip = ctrl.Antecedent(x, "food_quality_for_tip")
    service_quality_for_tip = ctrl.Antecedent(x, "service_quality_for_tip")

    tip = ctrl.Consequent(x_tip, "tip")

    var_map = {
        "temperature": temperature,
        "flavor": flavor,
        "portion_size": portion_size,
        "attentiveness": attentiveness,
        "friendliness": friendliness,
        "speed": speed,
        "food_quality": food_quality,
        "service_quality": service_quality,
        "food_quality_for_tip": food_quality_for_tip,
        "service_quality_for_tip": service_quality_for_tip,
        "tip": tip
    }

    idx = 0
    for name in VAR_NAMES:
        var = var_map[name]
        g1, g2, g3 = chromosome[idx:idx+3]
        idx += 3
        three_tri_from_genes(var, g1, g2, g3)

    # Food Quality (Stage 1) – Lab 4 rules
    rule1_food = ctrl.Rule(temperature["poor"] | flavor["poor"] | portion_size["poor"], food_quality["poor"])
    rule2_food = ctrl.Rule(temperature["good"] & flavor["good"], food_quality["good"])
    rule3_food = ctrl.Rule(flavor["good"] & portion_size["good"], food_quality["good"])
    rule4_food = ctrl.Rule(temperature["average"] & flavor["average"], food_quality["average"])
    rule5_food = ctrl.Rule(temperature["good"] & portion_size["average"], food_quality["average"])

    food_ctrl = ctrl.ControlSystem([rule1_food, rule2_food, rule3_food, rule4_food, rule5_food])

    # Service Quality (Stage 1) – Lab 4 rules
    rule1_service = ctrl.Rule(attentiveness["poor"] | friendliness["poor"] | speed["poor"], service_quality["poor"])
    rule2_service = ctrl.Rule(attentiveness["good"] & friendliness["good"], service_quality["good"])
    rule3_service = ctrl.Rule(friendliness["good"] & speed["good"], service_quality["good"])
    rule4_service = ctrl.Rule(attentiveness["average"] & friendliness["average"], service_quality["average"])
    rule5_service = ctrl.Rule(speed["good"] & attentiveness["average"], service_quality["average"])

    service_ctrl = ctrl.ControlSystem([rule1_service, rule2_service, rule3_service, rule4_service, rule5_service])

    # Tipper (Stage 2) – Lab 4 rules
    rule1_tip = ctrl.Rule(food_quality_for_tip["good"] & service_quality_for_tip["good"], tip["good"])
    rule2_tip = ctrl.Rule(food_quality_for_tip["poor"] & service_quality_for_tip["poor"], tip["poor"])
    rule3_tip = ctrl.Rule(food_quality_for_tip["good"] & service_quality_for_tip["average"], tip["average"])
    rule4_tip = ctrl.Rule(food_quality_for_tip["average"] & service_quality_for_tip["good"], tip["average"])
    rule5_tip = ctrl.Rule(food_quality_for_tip["average"] & service_quality_for_tip["average"], tip["average"])
    rule6_tip = ctrl.Rule(food_quality_for_tip["good"] & service_quality_for_tip["poor"], tip["average"])
    rule7_tip = ctrl.Rule(food_quality_for_tip["poor"] & service_quality_for_tip["good"], tip["average"])

    tip_ctrl = ctrl.ControlSystem([rule1_tip, rule2_tip, rule3_tip, rule4_tip, rule5_tip, rule6_tip, rule7_tip])

    return food_ctrl, service_ctrl, tip_ctrl

# run the fuzzy tree once
def execute_fuzzy_inference(food_ctrl, service_ctrl, tip_ctrl, inputs):
    food_sim = ctrl.ControlSystemSimulation(food_ctrl)
    food_sim.input["temperature"] = inputs["temperature"]
    food_sim.input["flavor"] = inputs["flavor"]
    food_sim.input["portion_size"] = inputs["portion_size"]
    food_sim.compute()
    fq = food_sim.output["food_quality"]

    service_sim = ctrl.ControlSystemSimulation(service_ctrl)
    service_sim.input["attentiveness"] = inputs["attentiveness"]
    service_sim.input["friendliness"] = inputs["friendliness"]
    service_sim.input["speed"] = inputs["speed"]
    service_sim.compute()
    sq = service_sim.output["service_quality"]

    tip_sim = ctrl.ControlSystemSimulation(tip_ctrl)
    tip_sim.input["food_quality_for_tip"] = fq
    tip_sim.input["service_quality_for_tip"] = sq
    tip_sim.compute()

    return tip_sim.output["tip"]

# evaluate one chromosome on a whole dataset
def evaluate_chromosome(chromosome, df):
    genes = extract_gene_values(chromosome)
    food_ctrl, service_ctrl, tip_ctrl = setup_fuzzy_system(genes)
    total_error = 0
    for index, row in df.iterrows():
        inputs = row_to_inputs(row)
        actual_tip = row["tip"]
        predicted_tip = execute_fuzzy_inference(food_ctrl, service_ctrl, tip_ctrl, inputs)
        error = abs(actual_tip - predicted_tip)
        total_error += error

    return total_error/len(df)

## Baseline chromosome and baseline MAE

In [ ]:
def make_baseline_chromosome():
    genes = []
    for i in VAR_NAMES:
        genes.extend([0.2, 0.5, 0.8])
    return genes

baseline_chromosome = make_baseline_chromosome()

baseline_train_mae = evaluate_chromosome(baseline_chromosome, train_df)
baseline_test_mae = evaluate_chromosome(baseline_chromosome, test_df)

print("Baseline (chromosome) train MAE:", baseline_train_mae)
print("Baseline (chromosome) test MAE:", baseline_test_mae)

Baseline (chromosome) train MAE: 5.673187027276372
Baseline (chromosome) test MAE: 5.049635346581666


## Fitness function and GA setup

In [ ]:
def fitness(chromosome):
    genes = extract_gene_values(chromosome)
    food_ctrl, service_ctrl, tip_ctrl = setup_fuzzy_system(genes)
    total_error = 0
    for index, row in train_df.iterrows():
        inputs = row_to_inputs(row)
        actual_tip = row["tip"]
        predicted_tip = execute_fuzzy_inference(food_ctrl, service_ctrl, tip_ctrl, inputs)
        error = abs(actual_tip - predicted_tip)
        total_error += error

    return total_error

ga = EasyGA.GA()

ga.chromosome_length = CHROMOSOME_LENGTH
ga.population_size = 20
ga.target_fitness_type = "min"
ga.generation_goal = 30
ga.gene_impl = lambda: random.random()
ga.fitness_function_impl = fitness

print("Evolving genetic fuzzy tree")
ga.evolve()
print("Evolution Done")
ga.print_best_chromosome()

Evolving genetic fuzzy tree
Evolution Done
Best Chromosome 	: [0.32350744953877386][0.4605608467923997][0.6136432512699393][0.3495414049549128][0.2625509791509355][0.43640372963853746][0.006464443407752207][0.39076795723290414][0.5451336709153833][0.3845659048243316][0.34332947365337185][0.24790799118755902][0.8572119243601359][0.8743297038415584][0.28172594008773677][0.3419802045378202][0.8209600018237236][0.14413305865393322][0.8099477603672084][0.3191075219467796][0.7328259351079034][0.6714107425730922][0.5111587386239405][0.7376556288598526][0.8052056092732175][0.17217391951489058][0.8198488280110626][0.6052815758675146][0.8494134869353582][0.19474990820764249][0.9592965268227848][0.31181006732514127][0.888546215195149]
Best Fitness    	: 749.9044493463027


## GA optimized MAE and improvement over baseline

In [ ]:
best_chrom = ga.population[0]
best_genes = extract_gene_values(best_chrom)

ga_test_mae = evaluate_chromosome(best_genes, test_df)

print("Baseline test MAE:", baseline_test_mae)
print("GA test MAE:", ga_test_mae)
print("Improvement:", baseline_test_mae - ga_test_mae)

Baseline test MAE: 5.049635346581666
GA test MAE: 3.8730573781781947
Improvement: 1.1765779684034712


## References:
- ECE 449 Lab 5 Presentation Slides
- ECE 449 Lab 5 PDF
- ECE 449 Lab 4 Mihir Mukhi
- ECE 449 Slides from Dr. Musilek
- skfuzzy Documentation: https://pythonhosted.org/scikit-fuzzy/auto_examples/index.html
- EasyGA Documentation: https://github.com/danielwilczak101/EasyGA